## Setup Environment

In [1]:
! pip install -qq unsloth==2025.4.1 vllm==0.7.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.2/193.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.9/396.9 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211

## Load Base Model

In [2]:
from vllm import SamplingParams
from unsloth import FastLanguageModel
from trl import GRPOConfig, GRPOTrainer

2025-04-29 14:13:11.639942: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745935991.884527      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745935991.961652      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_19/3848880211.py:2: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-29 14:13:34 __init__.py:207] Automatically detected platform cuda.


In [3]:
import os
import re
from datasets import load_dataset, Dataset

## Configure LoRA

- max_seq_length định nghĩa độ dài tối đa của input sequence.
- lora_rank xác định kích thước của ma trận adapter LoRA.
- load_in_4bit=False tắt lượng tử hóa để giữ độ chính xác cho reasoning.
- fast_inference=True tối ưu hóa throughput khi sinh text.
- gpu_memory_utilization=0.8 cho phép sử dụng tối đa 80% bộ nhớ GPU.

In [4]:
max_seq_length = 2048
lora_rank = 8

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="meta-llama/Llama-3.2-1B-Instruct",
    max_seq_length=max_seq_length,
    load_in_4bit=False,
    fast_inference=True,
    max_lora_rank=lora_rank,
    gpu_memory_utilization=0.8,
)


model = FastLanguageModel.get_peft_model(
    model,
    r=lora_rank,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=lora_rank,
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.1. vLLM: 0.7.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Llama-3.2-1B-Instruct with actual GPU utilization = 79.24%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 224.
Unsloth: vLLM's KV Cache can use up to 9.37 GB. Also swap space = 5 GB.
WARNING 04-29 14:13:40 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 04-29 14:13:52 config.py:549] This model supports multiple tasks: {'classify', 'embed', 'reward', 'generate', 'score'}. De

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

...

INFO 04-29 14:15:14 model_runner.py:1562] Graph capturing finished in 35 secs, took 0.20 GiB
INFO 04-29 14:15:14 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 57.38 seconds


tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth 2025.4.1 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


## Prepare Dataset

In [5]:
dataset = load_dataset("5CD-AI/Vietnamese-meta-math-MetaMathQA-40K-gg-translated", split="train")

README.md:   0%|          | 0.00/118 [00:00<?, ?B/s]

MetaMathQA-40K_vi.json:   0%|          | 0.00/69.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40000 [00:00<?, ? examples/s]

In [6]:
dataset[10]

{'response_vi': 'Để giải quyết vấn đề này, chúng ta cần xác định giá trị của x, đại diện cho số cuộn sợi màu xanh mà James có. Chúng ta hãy chia nhỏ thông tin đã cho: Số cuộn sợi màu đỏ: 12 Số cuộn sợi đen: 15 Số cuộn sợi màu xanh: x Số mũ nồi có thể làm được: 11 Chúng ta biết rằng James có thể làm một chiếc mũ nồi từ 3 những cuộn sợi. Do đó, tổng số cuộn sợi cần thiết cho 11 chiếc mũ nồi là 3 * 11 = 33. Ta lập phương trình như sau: Số cuộn sợi đỏ + Số cuộn sợi đen + Số cuộn sợi xanh = Tổng số cuộn sợi cần thiết 12 + 15 + x = 33 Hãy đơn giản hóa và giải x: 27 + x = 33 Để tách x, chúng ta trừ 27 từ cả hai vế của phương trình: 27 + x - 27 = 33 - 27 x = 6 Giá trị của x là 6. #### 6 Đáp án là: 6',
 'query_vi': 'James có thể làm một chiếc mũ nồi từ 3 cuộn sợi. Nếu anh ta có 12 cuộn chỉ đỏ, 15 cuộn chỉ đen và x cuộn chỉ xanh thì anh ta có thể làm được 11 chiếc mũ nồi.',
 'response_en': "To solve this problem, we need to determine the value of x, which represents the number of spools of blue 

In [7]:
answer_pattern = re.compile(
    # Sau đó cho phép có khoảng trắng tùy ý (\s*). -- bắt lấy bất kỳ nội dung nào phía sau ((.*)), chính là phần câu trả lời thực sự.
    r"(đáp án là:|đáp án là :|câu trả lời là:|câu trả lời là :)\s*(.*)",
    re.IGNORECASE
)

def extract__qanswer(example):
    # Tách câu hỏi và câu trả lời từ văn bản
    match = answer_pattern.search(example["response_vi"].strip().lower())
    question = example["query_vi"].strip().lower()
    if match:
        answer = match.group(2).strip()
    else:
        answer = ""
    return {"question": question, "answer": answer}


print(extract__qanswer(dataset[0]))

{'question': 'cha của reggie đã cho anh ấy 48 đô la. reggie đã mua 5 cuốn sách, mỗi cuốn có giá x. reggie còn lại 38 tiền. giá trị của biến x chưa biết là bao nhiêu?', 'answer': '2'}


In [8]:
standardized_dataset = dataset.map(
    extract__qanswer,
    remove_columns=dataset.column_names, # xóa các cột hiện tại (vì nó kh cần thiết)
    desc="Standardizing dataset",
).to_list()

Standardizing dataset:   0%|          | 0/40000 [00:00<?, ? examples/s]

In [9]:
standardized_dataset[0]

{'question': 'cha của reggie đã cho anh ấy 48 đô la. reggie đã mua 5 cuốn sách, mỗi cuốn có giá x. reggie còn lại 38 tiền. giá trị của biến x chưa biết là bao nhiêu?',
 'answer': '2'}

In [10]:
reasoning_start = "<thinking>"
reasoning_end = "</thinking>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

system_prompt = \
    f"""You are given a problem.
Think about the problem and provide your thought process.
Place it between {reasoning_start} and {reasoning_end}.
Then, provide your final answer between {solution_start}{solution_end}"""

train_dataset = Dataset.from_list(standardized_dataset[:2000]) 

# Khi map cần đặt đúng trường prompt và answers
train_dataset = train_dataset.map(lambda x: {
    "prompt": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": x["question"]},
    ],
    "answer": x["answer"],
})


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [11]:
train_dataset[2]

{'question': 'cally và danny giặt quần áo. cally có 10 áo sơ mi trắng, 5 áo sơ mi màu, 7 chiếc quần đùi và 6 chiếc quần dài, trong khi danny có 6 chiếc áo sơ mi trắng, 8 chiếc áo sơ mi màu, 10 chiếc quần đùi và 6 chiếc quần dài. họ đã giặt bao nhiêu bộ quần áo?',
 'answer': '58',
 'prompt': [{'content': 'You are given a problem.\nThink about the problem and provide your thought process.\nPlace it between <thinking> and </thinking>.\nThen, provide your final answer between <SOLUTION></SOLUTION>',
   'role': 'system'},
  {'content': 'cally và danny giặt quần áo. cally có 10 áo sơ mi trắng, 5 áo sơ mi màu, 7 chiếc quần đùi và 6 chiếc quần dài, trong khi danny có 6 chiếc áo sơ mi trắng, 8 chiếc áo sơ mi màu, 10 chiếc quần đùi và 6 chiếc quần dài. họ đã giặt bao nhiêu bộ quần áo?',
   'role': 'user'}]}

 ## Fine-tune LLM

### Before finetuning

In [12]:
def generate_response(model, tokenizer, train_dataset, idx=0, path_lora=None, system_prompt="You are a helpful assistant."):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": train_dataset[idx]["question"]},
    ]

    sampling_params = SamplingParams(
        temperature=0.8,
        top_p=0.95,
        max_tokens=1024,
    )

    text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
    )

    if path_lora != None:
        lora_request = model.load_lora(path_lora)
    else:
        lora_request = None
    output = model.fast_generate(
        [text],
        sampling_params=sampling_params,
        lora_request=lora_request,
    )[0].outputs[0].text

    print(f"Problem:\n{train_dataset[idx]['question']}")
    print(f"Response:\n{output}")
    print(f"GT Answer: {train_dataset[idx]['answer']}")

    return output
    
generate_response(model, tokenizer, train_dataset, idx=5, system_prompt = system_prompt)

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.52s/it, est. speed input: 58.28 toks/s, output: 57.88 toks/s]

Problem:
nếu simon hái 100 quả việt quất từ ​​bụi cây của mình và thêm 200 quả việt quất từ ​​bụi cây gần đó, và mỗi chiếc bánh cần 100 quả việt quất, simon có thể làm được bao nhiêu chiếc bánh việt quất?
Response:
<thinking>

Nếu Simon hái 100 quả việt quất từ bùi cây của mình và thêm 200 quả việt quất từ bùi cây gần đó, thì số lượng việt quất mà Simon có thể làm được là 100 + 200 = 300 quả.

Số lượng bánh việt quất cần làm là 300.

Để biết số lượng bánh việt quất Simon có thể làm được, ta chia số lượng việt quất cần làm bằng số lượng bánh việt quất mỗi chiếc bánh: 300 ÷ 100 = 3.

Vậy Simon có thể làm được 3 chiếc bánh việt quất.
GT Answer: 3


'<thinking>\n\nNếu Simon hái 100 quả việt quất từ bùi cây của mình và thêm 200 quả việt quất từ bùi cây gần đó, thì số lượng việt quất mà Simon có thể làm được là 100 + 200 = 300 quả.\n\nSố lượng bánh việt quất cần làm là 300.\n\nĐể biết số lượng bánh việt quất Simon có thể làm được, ta chia số lượng việt quất cần làm bằng số lượng bánh việt quất mỗi chiếc bánh: 300 ÷ 100 = 3.\n\nVậy Simon có thể làm được 3 chiếc bánh việt quất.'

### Reward function

Bắt đầu tìm từ solution_start (ví dụ <SOLUTION>).

- Sau đó:

    - Bỏ qua mọi ký tự sau <SOLUTION> (nhanh nhất có thể),

    - Tìm số đầu tiên (dãy [0-9], ,, . liền nhau),

    - Bắt dãy số đó vào 1 capture group

In [13]:
match_format = re.compile(
    rf"^[\s]{{0,}}"
    rf"{reasoning_start}.+?{reasoning_end}.*?"
    rf"{solution_start}(.+?){solution_end}"
    rf"[\s]{{0,}}\$",
    flags=re.MULTILINE | re.DOTALL
)

match_numbers = re.compile(
    solution_start + r".*?([\d\.\,]{1,})",
    flags=re.MULTILINE | re.DOTALL
)

test_samples = [
    "<SOLUTION> 42 </SOLUTION>",
    "<SOLUTION>\nThe answer is 3.14\n</SOLUTION>",
    "<SOLUTION> 1,234.56 something else </SOLUTION>",
    "<SOLUTION> abc def </SOLUTION>",    # không có số
    "Random text without solution tags",
]

# Test lần lượt từng mẫu
for idx, sample in enumerate(test_samples):
    match = match_numbers.search(sample)
    if match:
        print(f"Sample {idx}: FOUND number -> {match.group(1)}")
    else:
        print(f"Sample {idx}: No number found")

Sample 0: FOUND number -> 42
Sample 1: FOUND number -> 3.14
Sample 2: FOUND number -> 1,234.56
Sample 3: No number found
Sample 4: No number found


In [14]:

def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        if match_format.search(response) is not None:
            score += 3.0
        scores.append(score)
    return scores

def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        score += 0.5 if response.count(reasoning_start) == 1 else -1.0
        score += 0.5 if response.count(reasoning_end) == 1 else -1.0
        score += 0.5 if response.count(solution_start) == 1 else -1.0
        score += 0.5 if response.count(solution_end) == 1 else -1.0
        scores.append(score)
    return scores


In [15]:
def check_answer(prompts, completions, answer, **kwargs):
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1) if (guess := match_numbers.search(r)) is not None else None
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(0)
            continue
        if guess == true_answer:
            score += 3.0
        elif guess.strip() == true_answer.strip():
            score += 1.5
        else:
            score -= 1.5
        scores.append(score)
    return scores


def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None
        for r in responses
    ]
    
    # Lần đầu tiên bạn gọi check_numbers(), do check_numbers chưa có .counter → getattr(check_numbers, 'counter', 0) trả về 0, sau đó cộng 1 thành 1.
    count = getattr(check_numbers, 'counter', 0) + 1
    check_numbers.counter = count
    # if count % 5 == 0:
    #     print('*' * 20, f"Question:{question}",
    #           f"\nResponse:\n{responses[0]}",
    #           f"\nExtracted: {extracted_responses[0]}",
    #           f"\nGT Answer: {answer[0]}")

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(0)
            continue
        try:
            true_answer = float(true_answer.strip())
            guess = float(guess.strip().replace(",", ""))
            scores.append(1.5 if guess == true_answer else -0.5)
        except:
            scores.append(0)
    
    return scores

## Save Checkpoints

In [16]:
import os, wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Improving Reasoning with RL")

wandb.login(key=secret_value_0)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: moonlig73 (minhdeptrai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [17]:
data_tokenize = train_dataset.map(lambda x: {"tokens": tokenizer.apply_chat_template(x["prompt"], add_generation_prompt=True, tokenize=True)},batched=True,) # tokenize hết chattemplate
data_length = data_tokenize.map(lambda x: {"length" : len(x["tokens"])})
max_len = max(data_length["length"])
max_len

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

1015

In [18]:
max_prompt_length = max_len + 1

training_args = GRPOConfig(
    learning_rate=5e-6,
    weight_decay=5e-4,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="adamw_torch_fused",
    logging_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_generations=4,
    max_prompt_length=max_prompt_length,
    max_completion_length=max_seq_length - max_prompt_length, # Tổng độ dài prompt + completion không được vượt quá max_seq_length
    num_train_epochs=5,
    max_steps=-1,
    save_steps=250,
    max_grad_norm=0.1,
    report_to="wandb",
    output_dir="outputs_bz2",
)

trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
    ],
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 5 | Total steps = 625
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 8 x 1) = 64
 "-____-"     Trainable parameters = 5,636,096/1,241,450,496 (0.45% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Tracking run with wandb version 0.19.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250429_141544-sp28bmwv
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run outputs_bz2
wandb: ⭐️ View project at https://wandb.ai/minhdeptrai/huggingface
wandb: 🚀 View run at https:

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / match_format_exactly,rewards / match_format_approximately,rewards / check_answer,rewards / check_numbers
1,0.000000,-2.437500,0.457220,186.328125,0.000000,0.000000,-2.406250,-0.023438,-0.007812
2,0.000000,-2.609375,0.727760,264.187500,0.000000,0.000000,-2.523438,-0.070312,-0.015625
3,-0.000000,-2.828125,0.505673,212.796875,0.000000,0.000000,-2.828125,0.000000,0.000000
4,0.000000,-2.625000,0.566320,227.125000,0.000000,0.000000,-2.593750,-0.023438,-0.007812
5,0.000000,-2.523438,0.524102,208.828125,0.000016,0.000000,-2.523438,0.000000,0.000000
6,0.000000,-2.664062,0.683727,193.484375,0.000013,0.000000,-2.664062,0.000000,0.000000
7,0.000000,-2.664062,0.679887,202.234375,0.000016,0.000000,-2.734375,0.046875,0.023438
8,0.000000,-2.734375,0.574303,206.078125,0.000016,0.000000,-2.734375,0.000000,0.000000
9,0.000000,-2.695312,0.710717,179.296875,0.000017,0.000000,-2.664062,-0.023438,-0.007812
10,0.000000,-2.648438,0.524559,177.328125,0.000016,0.000000,-2.617188,-0.023438,-0.007812


TrainOutput(global_step=625, training_loss=0.004079509554832929, metrics={'train_runtime': 41634.2084, 'train_samples_per_second': 0.24, 'train_steps_per_second': 0.015, 'total_flos': 0.0, 'train_loss': 0.004079509554832929})

In [19]:
model.save_lora("grpo_saved_lora") # Lưu trong số adapter để inferences mình sẽ merge vào

## Run Evaluate

In [20]:
def generate_response(model, tokenizer, train_dataset, idx=0, path_lora=None, system_prompt="You are a helpful assistant."):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": train_dataset[idx]["question"]},
    ]

    sampling_params = SamplingParams(
        temperature=0.8,
        top_p=0.95,
        max_tokens=1024,
    )

    text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
    )

    if path_lora != None:
        lora_request = model.load_lora(path_lora)
    else:
        lora_request = None
    output = model.fast_generate(
        [text],
        sampling_params=sampling_params,
        lora_request=lora_request,
    )[0].outputs[0].text

    print(f"Problem:\n{train_dataset[idx]['question']}")
    print(f"Response:\n{output}")
    print(f"GT Answer: {train_dataset[idx]['answer']}")

    return output
    
generate_response(model, tokenizer, train_dataset, idx=5, path_lora="grpo_saved_lora" ,system_prompt = system_prompt)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.15s/it, est. speed input: 128.10 toks/s, output: 68.84 toks/s]

Problem:
nếu simon hái 100 quả việt quất từ ​​bụi cây của mình và thêm 200 quả việt quất từ ​​bụi cây gần đó, và mỗi chiếc bánh cần 100 quả việt quất, simon có thể làm được bao nhiêu chiếc bánh việt quất?
Response:
<thinking>Simon có 100 quả việt quất từ bùi cây của mình, sau đó thêm 200 quả việt quất từ bùi cây gần đó, tổng số lượng việt quất sẽ là 100 + 200 = 300</thinking>

Simon có thể làm được 300 quả việt quất</SOLUTION>
GT Answer: 3


'<thinking>Simon có 100 quả việt quất từ bùi cây của mình, sau đó thêm 200 quả việt quất từ bùi cây gần đó, tổng số lượng việt quất sẽ là 100 + 200 = 300</thinking>\n\nSimon có thể làm được 300 quả việt quất</SOLUTION>'